In [1]:
import os
import cv2
import pickle
from utils.dataset_utils import *

In [2]:
# Define paths
yolo_path = rf'..\models\costumized_yolo\costumized_yolo\costumized_yolo.pt' 
raw_video_folder = rf'..\data\1. Data Processing\raw\pred_prey_interaction' 
pred_attack_video_folder = rf'..\data\1. Data Processing\raw\pred_attack'
processed_video_folder = rf'..\data\1. Data Processing\processed\video'

In [3]:
video_count = 0
frame_count = 0
total_duration = 0

# loop over all raw videos
for video in os.listdir(raw_video_folder):
    # load video
    video_path = os.path.join(raw_video_folder, video)
    cap = cv2.VideoCapture(video_path)

    video_count += 1 # count videos

    # get properties
    frame_count += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = float(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # calculate duration
    duration = frame_count / fps
    total_duration += duration

# get format (hh:mm:ss)
total_seconds = int(total_duration)
hours = total_seconds // 3600
minutes = (total_seconds % 3600) // 60
seconds = total_seconds % 60

# print results
print(f'Number of Videos: {video_count}')
print(f'Total Duration: {hours}:{minutes}:{seconds}')
print(f'FPS: {fps}, Resolution: {width}x{height}')
print(f'Usable Frames: {frame_count}')

Number of Videos: 35
Total Duration: 16:42:49
FPS: 30.0, Resolution: 2160x2160
Usable Frames: 151695


In [12]:
video_count = 0
pred_attack_frame_count = 0

# loop over all raw videos
for video in os.listdir(pred_attack_video_folder):
    if video.endswith('.mp4'):
        # load video and get properties
        video_path = os.path.join(pred_attack_video_folder, video)
        cap = cv2.VideoCapture(video_path)

        video_count += 1 # count videos

        # get properties
        pred_attack_frame_count += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = float(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# calculate duration
total_attack_duration = pred_attack_frame_count / fps

total_seconds = int(total_attack_duration)
hours = total_seconds // 3600
minutes = (total_seconds % 3600) // 60
seconds = total_seconds % 60

# print results
print(f'Number Pred Attacks: {video_count}')
print(f'Total Attack Duration: {hours}:{minutes}:{seconds}')
print(f'FPS: {fps}, Resolution: {width}x{height}')
print(f'Usable Pred-Attack Frames: {pred_attack_frame_count}')
print(f'Attack vs. non-Attack Frame Ratio: {total_attack_duration / total_duration * 100:.2f}%')

Number Pred Attacks: 32
Total Attack Duration: 0:1:12
FPS: 30.0, Resolution: 2160x2160
Usable Pred-Attack Frames: 2183
Attack vs. non-Attack Frame Ratio: 0.12%


In [5]:
# window lengths to consider
window_lens = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# dict to save total windows per length
total_windows_per_length = {length: 0 for length in window_lens}

for video in os.listdir(raw_video_folder):
    # load filtered frames
    os.makedirs(os.path.join(processed_video_folder, "2. filtered_frames"), exist_ok=True)
    ff_path = os.path.join(processed_video_folder, "2. filtered_frames", f"filtered_frames_{video}.pkl")

    with open(ff_path, "rb") as f:
        filtered_frames = pickle.load(f)

    for length in window_lens:
        # find valid episodes and extract windows
        valid_episodes = find_valid_windows(filtered_frames, window_len=length, total_detections=33)
        extracted_windows = extract_windows(valid_episodes, window_len=length)
        total_windows_per_length[length] += len(extracted_windows) # accumulate total windows

for length in window_lens:
    print(f"Window Length {length} - Total Windows: {total_windows_per_length[length]}")

Window Length 1 - Total Windows: 11981
Window Length 2 - Total Windows: 3201
Window Length 3 - Total Windows: 1280
Window Length 4 - Total Windows: 628
Window Length 5 - Total Windows: 345
Window Length 6 - Total Windows: 195
Window Length 7 - Total Windows: 110
Window Length 8 - Total Windows: 68
Window Length 9 - Total Windows: 41
Window Length 10 - Total Windows: 24
Window Length 11 - Total Windows: 13
Window Length 12 - Total Windows: 6
Window Length 13 - Total Windows: 2
Window Length 14 - Total Windows: 1
Window Length 15 - Total Windows: 0


In [6]:
valid_frames = 11981
print(f"Ratio of frames with total detections (n=33): {valid_frames / frame_count * 100:.2f}%")

Ratio of frames with total detections (n=33): 7.79%
